In [ ]:
# +METO+

In [ ]:
#pip install --upgrade openai

In [ ]:
#pip install PyMuPDF

In [19]:
#funciona bien solo que no esta separando solo a textos importantes
import subprocess
from pathlib import Path
import fitz  # PyMuPDF
import re

pdf_path = Path("C:\\meto\\midocumento.pdf")

# Comprobar si el PDF existe
if pdf_path.exists():
    # Función para extraer texto del PDF
    def extraer_texto_pdf(pdf_path,pagina_inicio=2):
        texto = ""
        with fitz.open(pdf_path) as pdf:
            for pagina in pdf:
                # Comenzar a extraer desde la página especificada
                if pagina.number >= pagina_inicio:
                    texto += pagina.get_text() + "\n"
                 #  texto += "\n"  # Agregar un salto de línea entre páginas

        texto = texto.replace("...", "")  # Eliminar puntos suspensivos
        # Limpiar texto
        texto = ' '.join(texto.split())  # Reemplazar múltiples espacios por uno solo
                # Filtrar texto para obtener solo contenido relevante
        lineas_relevantes = []
        for linea in texto.split('\n'):
            if re.search(r'\b(importante|relevante|objetivo|política)\b', linea, re.IGNORECASE):
                lineas_relevantes.append(linea)

        # Unir las líneas relevantes
        texto_relevante = '\n'.join(lineas_relevantes)
        # Estructurar el texto relevante
        secciones = re.split(r'(\d+\.\s)', texto_relevante)  # Separar por secciones numeradas
        texto_organizado = ""

        for i in range(1, len(secciones), 2):  # Iterar por las secciones
            encabezado = secciones[i].strip()
            contenido = secciones[i + 1].strip() if i + 1 < len(secciones) else ""
            texto_organizado += f"{encabezado} {contenido}\n\n"

        return texto_organizado

    # Extraer el texto
    texto_pdf = extraer_texto_pdf(pdf_path, pagina_inicio=2)
    print(texto_pdf)

    # Imprimir longitud del texto
    print(f"Longitud del texto extraído: {len(texto_pdf)} caracteres")
    # Imprimir los primeros 100 caracteres
    print(f"Primeros 100 caracteres:\n{texto_pdf[:100]}")
else:
    print("El archivo PDF no existe en la ruta especificada.")


1. Propósito Describir los lineamientos para llevar a cabo el llenado de los expedientes para el “Círculo de Excelencia y Compromiso”.

2. Alcance Esta política aplica a todo el personal de la Organización.

3. Documentos de referencia • PERH-0114 Círculo de Excelencia y Compromiso. • PLSG-078 Gestión de Activos.

4. Definiciones • Expedientes: Conjunto de documentos, donde se describen a detalle de la o el nominada/o. • SPI: Término del proyecto en fecha. • CPI: Termino del proyecto en costo.

5. Política 5.

1. Llenado de Expedientes • Los expedientes deberán ser llenados de acuerdo a las características que tengan cada una de las categorías definidas en el procedimiento Círculo de Excelencia y Compromiso PERH-

0114. • Se requiere un mínimo de dos referencias por expediente, las cuales deben ser lo más explícitas posible, incluyendo los datos mínimos indispensables para cada expediente: o Nombre del Documento (Se coloca el nombre de la categoría correspondiente al expediente). o Obj

In [ ]:
#pip install PyMuPDF

In [ ]:
#pip install langchain

In [20]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import re
import fitz

def limpiar_texto(texto):
    # Elimina caracteres especiales, dejando solo letras, números y espacios
    texto = re.sub(r'[...]+', '', texto)  # Eliminar puntos suspensivos
    texto = re.sub(r'(\d+)\s+(\d+)', r'\1.\2', texto)
    texto = re.sub(r'[^a-zA-Z0-9áéíóúñÑ.\s./]', '', texto)
    return ' '.join(texto.split())

texto = ""
# Abre el documento PDF y extrae texto
with fitz.open(pdf_path) as doc:
    for pagina in doc:
        texto += pagina.get_text() + "\n"


texto_limpio = limpiar_texto(texto)
print(f"Longitud del texto limpio: {len(texto_limpio)}")

documents = [Document(page_content=texto_limpio)]
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30)

split_documents = text_splitter.split_documents(documents)#ok
#print(texto_limpio)
print(f"Longitud del texto original: {len(texto)}")
print(f"Número de chunks: {len(split_documents)}")

# Imprime los chunks
for i, chunk in enumerate(split_documents):
    print(f"Chunk {i}: {chunk.page_content}")


Longitud del texto limpio: 12915
Longitud del texto original: 15561
Número de chunks: 1
Chunk 0: Política Código PLRH0313 Elaboró o Modificó Heriberto lvarez Analista de Calidad Revisó Nancy Zepeda Gerencia de Recursos Humanos Aprobó Gabriela Santín Dirección de Negocio Círculo de Excelencia y Compromiso Circulo de Excelencia y Compromiso DI PLRH0313 Versión 4.18/12/2023 Clasificación Interna PGSG001FO02.2/11 Contenido 1 Propósito 3.2 Alcance 3.3 Documentos de referencia 3.4 Definiciones 3.5 Política 3.51 Llenado de Expedientes 3.52 Expedientes por Categoría 4.6 Formatos y Anexos 10.7 Control de versiones 11 Circulo de Excelencia y Compromiso DI PLRH0313 Versión 4.18/12/2023 Clasificación Interna PGSG001FO02.3/11.1 Propósito Describir los lineamientos para llevar a cabo el llenado de los expedientes para el Círculo de Excelencia y Compromiso 2 Alcance Esta política aplica a todo el personal de la Organización 3 Documentos de referencia PERH0114 Círculo de Excelencia y Compromiso PLSG07

## Divide documento en partes (*chunks*)

Esto cumple dos funciones:

 * Mejorar la relevancia semántica de nuestros embeddings: un documento muy grande puede cubrir demasiados temas, mientras que uno pequeño puede estar más enfocado en un solo tópico
 * Facilitar la tarea del modelo de lenuaje generativo – *chunks* más pequeños hacen que la ventana de contexto sea más pequeña

El proceso de división tiene varios parámetros: el tamaño del *chunk* y el tamaño de traslape entre *chunks*.

In [ ]:
#pip install -U langchain-community

In [21]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

embeddings = HuggingFaceBgeEmbeddings()
query_result = embeddings.embed_query(split_documents[0].page_content)
query_result

c:\Codex\IAPRAXTHON2024\iadjango1\env\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


[0.012197311036288738,
 -0.011831606738269329,
 0.016790280118584633,
 0.021407825872302055,
 -0.017280060797929764,
 -0.02477354183793068,
 -0.026713494211435318,
 0.004542400129139423,
 0.0033425302244722843,
 0.004807918332517147,
 0.05093127861618996,
 -0.010065565817058086,
 0.011330874636769295,
 -0.021821841597557068,
 -0.033488113433122635,
 0.02354242280125618,
 -0.021389203146100044,
 -0.02699589729309082,
 -0.005587894469499588,
 0.012693732976913452,
 0.0013639491517096758,
 0.00484292209148407,
 -0.09033430367708206,
 -0.01703433133661747,
 -0.03637992590665817,
 0.06205551698803902,
 0.01647045463323593,
 0.006666922476142645,
 0.07546966522932053,
 0.039894506335258484,
 -0.033918220549821854,
 -0.015647193416953087,
 0.011758607812225819,
 -0.04560677707195282,
 0.00856736209243536,
 -0.0029282723553478718,
 0.02439585141837597,
 -0.027653904631733894,
 -0.002527872333303094,
 -0.02484694868326187,
 0.03068496286869049,
 -0.001515399431809783,
 0.0032102810218930244,
 -

## Guardamos a la base de datos tipo vectorial Chroma

In [22]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(split_documents, embeddings)

# Verifica si el vectorstore se ha creado correctamente
print(f"Vectorstore creado con {len(vectorstore)} vectores.")

Vectorstore creado con 2 vectores.


In [23]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from transformers import pipeline  # Importa Hugging Face
from langchain_community.llms import Ollama

# Inicializar el modelo Llama 3
llm = Ollama(model="llama3", temperature=0.1)

qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(),
   # return_source_document = True
)

In [24]:
chat_history = []
cache = {}
#query = "Por favor, responde únicamente en español: Dime los expedientes por categoria por aportaciones a Praxis"
query = "Por favor, responde únicamente en español: Dime los Expedientes por Categoría, que corresponde al documento Circulo de excelencia y compromiso PLRH-0313"

# Verificar si la consulta ya está en el caché
if query in cache:
    respuesta = cache[query]
    print("Respuesta desde caché:", respuesta)
else:
    # Si no está en el caché, hacer la consulta al modelo
    inputs = {
        "question": query,
        "chat_history": chat_history
    }
    print(inputs) 

    result = qa(inputs)
    print("Resultado completo:", result)  # Imprime el resultado completo para inspección
#result = qa({"Question": query, "chat_history": chat_history})
if "answer" in result:
    respuesta = result["answer"]
    print("Respuesta:", respuesta)
    # Actualizar el historial de chat
    chat_history.append((query, respuesta))
else:
    print("No se obtuvo respuesta.")

{'question': 'Por favor, responde únicamente en español: Dime los Expedientes por Categoría, que corresponde al documento Circulo de excelencia y compromiso PLRH-0313', 'chat_history': []}


Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


Resultado completo: {'question': 'Por favor, responde únicamente en español: Dime los Expedientes por Categoría, que corresponde al documento Circulo de excelencia y compromiso PLRH-0313', 'chat_history': [], 'answer': 'Según el contenido del documento "Círculo de Excelencia y Compromiso DI PLRH0313 Versión 4 18122023", los expedientes por categoría son:\n\n1. A la Innovación (10)\n2. Al dominio y expertise de negocios (11)\n3. A la trayectoria profesional (12)\n\nNo hay otros expedientes mencionados en el documento.'}
Respuesta: Según el contenido del documento "Círculo de Excelencia y Compromiso DI PLRH0313 Versión 4 18122023", los expedientes por categoría son:

1. A la Innovación (10)
2. Al dominio y expertise de negocios (11)
3. A la trayectoria profesional (12)

No hay otros expedientes mencionados en el documento.


In [ ]:
# Mostrar el historial de chat
print("Historial de chat:")
for pregunta, respuesta in chat_history:
    print(f"Pregunta: {pregunta}\nRespuesta: {respuesta}\n")